# AdImpactSimulation

This is a simple simulation of the impact of brand and targeted direct response advertising
on purchase decisions.

## User Needs

People (users) have "needs" for products and services which change over time.
Needs for products that are consumed regularly, like food and shampoo, increase as
existing products are consumed.  Needs for other products, like new electronic devices,
only develop as consumers learn about the products and how they might be useful.
Advertising can help make people aware of new products and ways that they might be helpful,
and this can increase the need for those products.

We'll start by using the same needs for all users.  Later, we might want different
levels of needs for different demographic groups.

Our similation will look at needs and advertising over time in weekly units of time.


In [1]:
# Time unit
time_unit_name = "week"

# General time-based need for a product.
# A typical user's need will increase by about this much each week, regardless on advertising.
time_based_need = 1 / 52

# General brand advertising-based need for a product.
# One ad for a product in the past week will result in this much need for a typical user
# who may be likely to purchase the product.
# (Some products are only appropriate for a subset of the population.)
brand_based_need = 1 / 100

## Brands

The simulations will assume that there are multiple brands producing competing products.
Each brand can have difference levels of brand and direct response advertising and
different search rankings (which might include natural or paid search).

We'll start by using the same advertising levels for all users.  Later, we might want different
levels of advertising for different demographic groups.

In [2]:
import random

# The number of advertisers / product manufacturers in the simulation
advertiser_count = 4

# The names of the advertisers / product manufacturers
advertiser_names = ["Advertiser " + str(i + 1) for i in range(advertiser_count)]

# The expected number of brand ads from each advertiser in a week
advertiser_brand_frequency = {
    adv : random.triangular(low = 0.2, high = 2.0, mode = 1.0)
    for adv in advertiser_names
}

# The expected number of trageted ads (to targeted users) from each advertiser in a week
advertiser_targeted_frequency = {
    adv : random.triangular(low = 0.2, high = 2.0, mode = 1.0)
    for adv in advertiser_names
}

# The search positions of the advertisers
advertiser_search_position = { adv : i + 1 for i, adv in enumerate(advertiser_names)}

# Brand dimensions
# Some users prefer low price or high quality, ease of use, or other factors.
# Users will be more likely to buy products that match the values that are most important to them.
brand_dimensions = ['Price', 'Quality', 'Ease of Use']
advertiser_brand_dimension_ratings = {
    adv : {
        brand_dim : random.random()
        for brand_dim in brand_dimensions
    }
    for adv in advertiser_names
}

## Advertising Impact

The impact of advertising diminishes as advertising increases and over time.
For example, the impact of 10 ads to a user in a week may be less than 10 times
the impact of 1 ad to a user in a week, and the impact of an ad 2 weeks ago is probably
less than the impact of an ad 1 week ago.

We use saturation functions to model the weekly ad saturation effect.  The decrease in ad
impact over time is modeled by an exponential decay with a half-life that specifies the number
of weeks when impact of the ads will decrease by one half.

In [3]:
# General sturation function
# No saturation when the weekly ad count is not more than saturation_start.
# Saturation with additional ads so that the total returned is never more than
# saturation_start + saturation_range
def ad_saturation(ad_count, saturation_start = 4, saturation_range = 6):
    if ad_count <= saturation_start:
        return ad_count
    adjusted_ad_count = (ad_count - saturation_start) / saturation_range
    return saturation_start + saturation_range * (adjusted_ad_count / (adjusted_ad_count + 1))


# Brand sturation function
# No saturation when the weekly ad count is not more than saturation_start.
# Saturation with additional ads so that the total returned is never more than
# saturation_start + saturation_range
def weekly_brand_saturation(ad_count, saturation_start = 4, saturation_range = 6):
    return ad_saturation(
        ad_count,
        saturation_start = saturation_start,
        saturation_range = saturation_range
    )

# Targerted sturation function
# No saturation when the weekly ad count is not more than saturation_start.
# Saturation with additional ads so that the total returned is never more than
# saturation_start + saturation_range
def weekly_targeted_saturation(ad_count, saturation_start = 4, saturation_range = 6):
    return ad_saturation(
        ad_count,
        saturation_start = saturation_start,
        saturation_range = saturation_range
    )


# Brand half-life
# Brand messages are half as effective after this many weeks
brand_half_life_weeks = 12

# Targerted half-life
# Targerted messages are half as effective after this many weeks
targeted_half_life_weeks = 2

## Users

Users have:
* Preferences for certain types of products.  These include brand dimension preferences.
* A need that changes over time.  Some users will never have a need for some products.
* Per-advertiser advertising impacts that vary over time.
* A history of past product research, searches, site visits, and purchases.
* Product preferences based on user and advertiser attributes.
* Probabilities of taking actions like searchs and purchases in the next time period.

In [5]:
class AdUser:
    def __init__(self):
        # User attributes, including preferences
        
        # avg ad opportunities in a period
        self.avg_period_ad_opportunities = 20
        
        # Brand dimension preferences
        self.brand_dimension_preferences = {
            brand_dim : random.random()
            for brand_dim in brand_dimensions
        }
        
        
        # Time-varying properties
        
        # Time-based need
        self.time_based_need = 0
        
        
        # Per-advertiser attributes
        self.adv_brand_impact = {}
        self.adv_targeted_impact = {}
        self.adv_research_impact = {}
        
        
        # State
        
        # Last Times (we may want some of these to be by advertiser)
        self.last_research_time = None
        self.last_site_visit = None
        self.last_purchase = None
        
    def CurrentNeed(self):
        raise NotImplemented()
         
    def IsTargetable(self):
        raise NotImplemented()
        
    def ResearchProbability(self):
        raise NotImplemented()

    def PurchaseProbability(self):
        raise NotImplemented()
        
    def BrandFavorability(self, adv):
        raise NotImplemented()
        
    def SiteVisitProbability(self, adv):
        raise NotImplemented()
  

## Events in a period of the Simulation

For each user in each period of ths simulation:
* The user can get brand ads
    * The probability is Poisson by advertiser based on user ad opportunities and expected adv frequency for an average user
    * The impact may depend on the brand dimension match between the advertiser and user
* The user may get targeted ads if they are targetable
    * They're generally targetable if they've done research recently
    * Targetable to all advertisers, since data is shared
    * More likely to be targeted if they visited an advertiser's site recently
    * The user may respond to targeted ads by visiting an advertiser's site
* The user may do product research
    * The probability increase with need (maybe proportional to need squared)
    * The user may do searches or visit sites when doing research
        * Site visits add to direct impact (for example, one visit might be similar to 5 targeted ads)
        * Adversiers with high brand impact and matching brand dimensions are more likely to get site visits
        * Visits from searches depend on those factors and search position
        * Past site visits and favorable research impacts make future site visites more likely
    * Research and site visits might give favorable or unfavorable research impacts
* The user may make a purchase
    * Purchases will only be made after site visits in the same period
    * Purchase probability increases with need